# Optimize Initial Conditions
## Yabox

In [1]:
from scipy.optimize import brute
import numpy as np
from datetime import datetime,timedelta
import pandas as pd
from yabox import DE

# Initialize Ray

In [2]:
from environs import Env
env = Env()
env.str("CUDA_DEVICE_ORDER",'PCI_BUS_ID')
env.str("CUDA_VISIBLE_DEVICES","96")
env.int("NUMBA_ENABLE_CUDASIM",1)
env.bool("OMPI_MCA_opal_cuda_support",True)

import os
import ray
MB=1024*1024
GB=MB*1024
ray.shutdown()
ray.init(num_gpus=96,num_cpus=17,
         ignore_reinit_error=True)

@ray.remote(num_gpus=1)
def use_gpu():
    print("ray.get_gpu_ids(): {}".format(ray.get_gpu_ids()[0]))
    print("CUDA_VISIBLE_DEVICES: {}".format(os.environ["CUDA_VISIBLE_DEVICES"]))

use_gpu.remote()

2021-03-02 09:57:03,946	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265


ObjectRef(df5a1a828c9685d3ffffffff0100000001000000)

# Load New and Process Data from website data.brasil.io

In [3]:
%reload_ext autoreload
%autoreload 2
import get_data as gd
LoadData=False

if LoadData:
    gd.get_data()

# Functions to Load Processed Data

In [4]:
def load_confirmed(districtRegion, start_date):
    dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')
    df = pd.read_csv('./data/DRS_confirmados.csv',delimiter=',',parse_dates=True, date_parser=dateparse)
    y=[]
    x=[]
    start=datetime.strptime(start_date, "%Y-%m-%d")+timedelta(days=40)
    start2=start.strftime("%Y-%m-%d")
    for i in range(0,len(df.date)):
        y.append(df[districtRegion].values[i])
        x.append(df.date.values[i])
    df2=pd.DataFrame(data=y,index=x,columns=[""])
    df2 =df2.apply (pd.to_numeric, errors='coerce')
    df2[start2:] = df2[start2:].replace({0:np.nan})
    df2 = df2.dropna()
    df2.index = pd.DatetimeIndex(df2.index)
    #interpolate missing data
    df2 = df2.reindex(pd.date_range(df2.index.min(), df2.index.max()), fill_value=np.nan)
    df2 = df2.interpolate(method='akima', axis=0).ffill().bfill()
    #string type for dates and integer for data
    df2 = df2.astype(int)
    df2.index = df2.index.astype(str)
    #select dates
    df2=df2[start_date:]
    del x,y,df,dateparse
    return df2

def load_dead(districtRegion, start_date):
    dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')
    df = pd.read_csv('./data/DRS_mortes.csv',delimiter=',',parse_dates=True, date_parser=dateparse)
    y=[]
    x=[]
    start=datetime.strptime(start_date, "%Y-%m-%d")+timedelta(days=40)
    start2=start.strftime("%Y-%m-%d")
    for i in range(0,len(df.date)):
        y.append(df[districtRegion].values[i])
        x.append(df.date.values[i])
    df2=pd.DataFrame(data=y,index=x,columns=[""])
    df2 =df2.apply (pd.to_numeric, errors='coerce')
    df2[start2:] = df2[start2:].replace({0:np.nan})
    df2 = df2.dropna()
    df2.index = pd.DatetimeIndex(df2.index)
    #interpolate missing data
    df2 = df2.reindex(pd.date_range(df2.index.min(), df2.index.max()), fill_value=np.nan)
    df2 = df2.interpolate(method='akima', axis=0).ffill().bfill()
    #string type for dates and integer for data
    df2 = df2.astype(int)
    df2.index = df2.index.astype(str)
    #select dates
    df2=df2[start_date:]
    del x,y,df,dateparse
    return df2

(pid=8915) ray.get_gpu_ids(): 95
(pid=8915) CUDA_VISIBLE_DEVICES: 95


# Load solver

In [5]:
import LearnerICRayNoLoadBHNewModel_v4 as L 

# Data for Regions

In [13]:
dfparam = pd.read_csv("data/param.csv")
dfPopEst = pd.read_csv("data/DRS_popEst.csv")
dfparam['popEst']=dfPopEst.popEst
dfparam['RATIO']=0
display(dfparam)


,DRS,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,RATIO,WCASES,WREC,popEst
0,DRS 01 - Grande São Paulo,2020-03-20,200,2000000.0,0.0001,0.0001,800.0000,300.0000,400.0000,0,0,0.15,0.05,21734682.0
1,DRS 02 - Araçatuba,2020-04-01,200,500.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0,0.40,0.10,791256.0
2,DRS 03 - Araraquara,2020-04-01,200,2000.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0,0.40,0.10,1025982.0
3,DRS 04 - Baixada Santista,2020-04-01,200,8000.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0,0.40,0.10,1865397.0
4,DRS 05 - Barretos,2020-04-01,200,2000.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0,0.40,0.10,440907.0
5,DRS 06 - Bauru,2020-04-01,200,10000.0,0.0001,0.0001,4.0000,0.0001,0.0001,0,0,0.40,0.10,1800757.0
6,DRS 07 - Campinas,2020-04-01,200,20000.0,0.0001,0.0001,40.0000,0.0001,0.0001,0,0,0.40,0.10,4671287.0
7,DRS 08 - Franca,2020-04-01,200,1000.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0,0.40,0.10,718176.0
8,DRS 09 - Marília,2020-04-01,200,5000.0,0.0001,0.0001,0.0001,0.0001,0.0001,0,0,0.40,0.10,1149132.0
9,DRS 10 - Piracicaba,2020-04-01,200,10000.0,0.0001,0.0001,0.0001,0.0001,1.0000,0,0,0.40,0.10,1586546.0


In [7]:
dfSP = pd.read_csv("data/dados_municipios_SP.csv")

# lista DRSs
DRS = list(dfSP["DRS"].unique())
DRS.remove("Indefinido")

dfSP

,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,popEst,estimated_population,city_ibge_code,confirmed_per_100k_inhabitants,death_rate,DRS
0,2021-02-01,SP,TOTAL,state,1779722,53090,336,True,45919049.0,46289333.0,35.0,3844.77780,0.0298,Indefinido
1,2021-01-31,SP,TOTAL,state,1777368,53034,335,False,45919049.0,46289333.0,35.0,3839.69240,0.0298,Indefinido
2,2021-01-30,SP,TOTAL,state,1773024,52954,334,False,45919049.0,46289333.0,35.0,3830.30795,0.0299,Indefinido
3,2021-01-29,SP,TOTAL,state,1759957,52722,333,False,45919049.0,46289333.0,35.0,3802.07898,0.0300,Indefinido
4,2021-01-28,SP,TOTAL,state,1746070,52481,332,False,45919049.0,46289333.0,35.0,3772.07855,0.0301,Indefinido
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155027,2020-06-23,SP,Óleo,city,1,0,5,False,2496.0,2471.0,3533809.0,40.46945,0.0000,DRS 09 - Marília
155028,2020-06-22,SP,Óleo,city,1,0,4,False,2496.0,2471.0,3533809.0,40.46945,0.0000,DRS 09 - Marília
155029,2020-06-21,SP,Óleo,city,1,0,3,False,2496.0,2471.0,3533809.0,40.46945,0.0000,DRS 09 - Marília
155030,2020-06-20,SP,Óleo,city,1,0,2,False,2496.0,2471.0,3533809.0,40.46945,0.0000,DRS 09 - Marília


# Functions for Optimization

In [8]:
from scipy.integrate import odeint
import sys
import io
import gc

def create_f(districtRegion,e0,a0,r0,date, ratio, predict_range, version):
                
    def fobjective(point):
        
        cleanRecovered=False
        s0, deltaDate, i0, d0, startNCases, weigthCases = point
        weigthRecov=0
        ratio=0
        dead = load_dead(districtRegion,date)
        data = load_confirmed(districtRegion,date)*(1-ratio)-dead
        end_date=datetime.strptime(date, "%Y-%m-%d") + timedelta(days=deltaDate)
        f=L.Learner.remote(districtRegion, end_date.strftime("%Y-%m-%d"), predict_range,\
                           s0, e0, a0, i0, r0, d0, startNCases, ratio, weigthCases, weigthRecov, \
                           cleanRecovered, version, data, dead, savedata=False)
        result = f.train.remote() 
        result = ray.get(result) 

        del end_date,cleanRecovered, point,f         

        gc.collect()

        return result
    return fobjective

In [9]:
@ray.remote(num_cpus=0,num_gpus=4) #, max_calls=1)
def opt(districtRegion,s0,i0,e0,a0,r0,d0,wcases,date,startNCases, 
        ratio, predict_range, popEst, version):

    bounds=[(0.3*popEst,0.99*popEst),(-4,4),(0,250), (0,250), (0,250),\
              (0.19,0.85)]
    maxiterations=500
    runDate=datetime.today()
    f=create_f(districtRegion,e0,a0,r0,date, ratio, predict_range, version)
    de = DE(f, bounds, maxiters=maxiterations)
    for step in de.geniterator():
        try:
            idx = step.best_idx
            norm_vector = step.population[idx]
            best_params = de.denormalize([norm_vector])
            del norm_vector, idx
        except:
            print("error in function evaluation")
    p=best_params[0]
    
    return p

# Main Code

In [10]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return true;}

<IPython.core.display.Javascript object>

In [11]:
DRS=dfparam.DRS
display(DRS)
version="119"
gc.enable()

optimal=[]
for districtRegion in DRS:
    #remove previous history file
    strFile='./results/history_'+districtRegion+version+'.csv'
    if os.path.isfile(strFile):
        os.remove(strFile)
    query = dfparam.query('DRS == "{}"'.format(districtRegion)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    date,predict_range,s0,e0,a0,i0,r0,d0,startNCases,ratio,wcases,wrec,popEst = parameters
    optimal.append(opt.remote(districtRegion,s0,i0,e0,a0,r0,d0,wcases,date,startNCases, \
                              ratio, predict_range, popEst, version))        
    

0          DRS 01 - Grande São Paulo
1                 DRS 02 - Araçatuba
2                DRS 03 - Araraquara
3          DRS 04 - Baixada Santista
4                  DRS 05 - Barretos
5                     DRS 06 - Bauru
6                  DRS 07 - Campinas
7                    DRS 08 - Franca
8                   DRS 09 - Marília
9                DRS 10 - Piracicaba
10      DRS 11 - Presidente Prudente
11                 DRS 12 - Registro
12           DRS 13 - Ribeirão Preto
13    DRS 14 - São João da Boa Vista
14    DRS 15 - São José do Rio Preto
15                 DRS 16 - Sorocaba
16                  DRS 17 - Taubaté
Name: DRS, dtype: object

In [ ]:
optimal=ray.get(optimal)

(pid=9251) basinhopping step 0: f 1.45534e+08


(pid=9251) /home/ats4i/anaconda3/envs/geo_env/lib/python3.6/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Repeated error test failures (internal error). Run with full_output = 1 to get quantitative information.
(pid=9251)   warnings.warn(warning_msg, ODEintWarning)


(pid=9260) basinhopping step 0: f 2.26729e+09
(pid=9252) basinhopping step 0: f 2.56199e+08
(pid=9252) warning: basinhopping: local minimization failure
(pid=9252) basinhopping step 1: f 2.56199e+08 trial_f 2.59943e+08 accepted 0  lowest_f 2.56199e+08
(pid=9255) basinhopping step 0: f 1.73339e+09
(pid=9255) basinhopping step 1: f 1.73339e+09 trial_f 1.75381e+09 accepted 0  lowest_f 1.73339e+09
(pid=9259) warning: basinhopping: local minimization failure
(pid=9259) basinhopping step 0: f 9.6841e+08
(pid=9252) basinhopping step 2: f 2.56199e+08 trial_f 2.57881e+08 accepted 0  lowest_f 2.56199e+08
(pid=9260) warning: basinhopping: local minimization failure
(pid=9260) basinhopping step 1: f 1.69614e+09 trial_f 1.69614e+09 accepted 1  lowest_f 1.69614e+09
(pid=9260) found new global minimum on step 1 with function value 1.69614e+09
(pid=9255) warning: basinhopping: local minimization failure
(pid=9255) basinhopping step 2: f 1.73339e+09 trial_f 1.74856e+09 accepted 0  lowest_f 1.73339e+09


(pid=9257) /home/ats4i/anaconda3/envs/geo_env/lib/python3.6/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Repeated error test failures (internal error). Run with full_output = 1 to get quantitative information.
(pid=9257)   warnings.warn(warning_msg, ODEintWarning)


(pid=9253) warning: basinhopping: local minimization failure
(pid=9253) basinhopping step 10: f 3.67365e+07 trial_f 2.95448e+11 accepted 0  lowest_f 3.67365e+07
(pid=9257) warning: basinhopping: local minimization failure
(pid=9257) basinhopping step 8: f 5.51636e+08 trial_f 5.98727e+08 accepted 0  lowest_f 5.51636e+08
(pid=9251) warning: basinhopping: local minimization failure
(pid=9251) basinhopping step 3: f 7.10851e+07 trial_f 7.10851e+07 accepted 1  lowest_f 7.10851e+07
(pid=9251) found new global minimum on step 3 with function value 7.10851e+07
(pid=9257) basinhopping step 9: f 5.51636e+08 trial_f 6.03569e+08 accepted 0  lowest_f 5.51636e+08
(pid=9251) basinhopping step 4: f 7.10851e+07 trial_f 1.4768e+08 accepted 0  lowest_f 7.10851e+07
(pid=9257) basinhopping step 10: f 5.51636e+08 trial_f 6.03569e+08 accepted 0  lowest_f 5.51636e+08
(pid=10198) warning: basinhopping: local minimization failure
(pid=10198) basinhopping step 0: f 3.96849e+10
(pid=10198) basinhopping step 1: f 

...

(pid=20747) basinhopping step 1: f 9.23092e+08 trial_f 9.31643e+08 accepted 0  lowest_f 9.23092e+08
(pid=19892)  lsoda--  at t(=r1) and step size h(=r2), the error  
(pid=19892)        test failed repeatedly or with abs(h) = hmin  
(pid=19892)       in above,  r1 =  0.1215907305373D+03   r2 =  0.1000000000000D-11
(pid=19892)  lsoda--  at t(=r1) and step size h(=r2), the error  
(pid=19892)        test failed repeatedly or with abs(h) = hmin  
(pid=19892)       in above,  r1 =  0.1215697240307D+03   r2 =  0.1000000000000D-11
(pid=19892)  lsoda--  at t(=r1) and step size h(=r2), the error  
(pid=19892)        test failed repeatedly or with abs(h) = hmin  
(pid=19892)       in above,  r1 =  0.1215907305373D+03   r2 =  0.1000000000000D-11
(pid=19892)  lsoda--  at t(=r1) and step size h(=r2), the error  
(pid=19892)        test failed repeatedly or with abs(h) = hmin  
(pid=19892)       in above,  r1 =  0.1215907303649D+03   r2 =  0.1000000000000D-11
(pid=19892)  lsoda--  at t(=r1) and step

(pid=19892) basinhopping step 3: f 1.24841e+08 trial_f 1.43338e+11 accepted 0  lowest_f 1.24841e+08
(pid=20747) warning: basinhopping: local minimization failure
(pid=20747) basinhopping step 2: f 9.23092e+08 trial_f 9.26923e+08 accepted 0  lowest_f 9.23092e+08
(pid=20316) basinhopping step 0: f 1.43369e+11
(pid=18975) warning: basinhopping: local minimization failure
(pid=18975) basinhopping step 8: f 3.48212e+09 trial_f 9.84448e+11 accepted 0  lowest_f 3.48212e+09
(pid=20694) warning: basinhopping: local minimization failure
(pid=20694) basinhopping step 1: f 1.46558e+09 trial_f 2.08744e+09 accepted 0  lowest_f 1.46558e+09
(pid=19892) warning: basinhopping: local minimization failure
(pid=19892) basinhopping step 4: f 1.24841e+08 trial_f 1.47097e+08 accepted 0  lowest_f 1.24841e+08
(pid=20694) basinhopping step 2: f 1.46558e+09 trial_f 1.50311e+09 accepted 0  lowest_f 1.46558e+09
(pid=19892) basinhopping step 5: f 1.24841e+08 trial_f 1.47097e+08 accepted 0  lowest_f 1.24841e+08
(pid=

(pid=20694) /home/ats4i/anaconda3/envs/geo_env/lib/python3.6/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Repeated error test failures (internal error). Run with full_output = 1 to get quantitative information.
(pid=20694)   warnings.warn(warning_msg, ODEintWarning)


(pid=20747) basinhopping step 5: f 1.64497e+07 trial_f 1.64497e+07 accepted 1  lowest_f 1.64497e+07
(pid=20747) found new global minimum on step 5 with function value 1.64497e+07
(pid=21488) basinhopping step 0: f 5.57227e+07
(pid=20694) basinhopping step 6: f 1.46558e+09 trial_f 1.49501e+09 accepted 0  lowest_f 1.46558e+09
(pid=21055) warning: basinhopping: local minimization failure
(pid=21055) basinhopping step 4: f 1.28341e+08 trial_f 2.28054e+08 accepted 0  lowest_f 1.28341e+08
(pid=20316) basinhopping step 3: f 1.43369e+11 trial_f 1.17493e+13 accepted 0  lowest_f 1.43369e+11
(pid=20694) basinhopping step 7: f 1.46558e+09 trial_f 1.48888e+09 accepted 0  lowest_f 1.46558e+09
(pid=20747) basinhopping step 6: f 1.64497e+07 trial_f 1.83725e+09 accepted 0  lowest_f 1.64497e+07
(pid=20694) basinhopping step 8: f 1.46558e+09 trial_f 1.50304e+09 accepted 0  lowest_f 1.46558e+09
(pid=21055) warning: basinhopping: local minimization failure
(pid=21055) basinhopping step 5: f 1.28341e+08 tri

(pid=21055) /home/ats4i/anaconda3/envs/geo_env/lib/python3.6/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Repeated error test failures (internal error). Run with full_output = 1 to get quantitative information.
(pid=21055)   warnings.warn(warning_msg, ODEintWarning)


In [ ]:
for i in range(0,len(DRS)):    

    districtRegion=DRS[i]
    query = dfparam.query('DRS == "{}"'.format(districtRegion)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    startdate,predict_range,s0,e0,a0,i0,r0,d0,startNCases,ratio,wcases,wrec = parameters
    Date = datetime.strptime(startdate, "%Y-%m-%d")
    end_date = Date + timedelta(days=+int(optimal[i][1]))
    dateStr=end_date.strftime("%Y-%m-%d")
    
    #s0, deltaDate, i0, d0, startNCases, weigthCases
    j = query['index'].values[0]
    dfparam.at[j, "s0"] = optimal[i][0]
    dfparam.at[j, "start-date"] = dateStr
    dfparam.at[j, "i0"] = optimal[i][2]
    dfparam.at[j, "d0"] = optimal[i][3]
    dfparam.at[j, "START"] = optimal[i][4]
    dfparam.at[j, "WCASES"] = optimal[i][5]

    dfparam.to_csv("data/param_optimized_Yabox.csv", sep=",", index=False)
    dfparam
    